# League of Legends (LoL) Neural Network for Masters+ Games

This notebook builds and trains a neural network to predict win/loss outcomes for LoL Masters+ games.

In [1]:
from lol_scrape_tools import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from aitk.utils import gallery, array_to_image
from aitk.networks import Network

# Get the data
* Download the data (champion data, game data) from Data folder
* Use champion data to transform game data into neural network inputs and outputs

In [2]:
champions_df=pd.read_csv('Data/Champion_Info (Masters+).csv')
champ_to_info=champToInfo(champions_df)

# uncomment to use version 2
# champ_to_info=champToInfo(champions_df,version=2)

In [3]:
champ_to_info['Aatrox']

[0.4641, 'X', 0.5257, 'X', 'X']

In [4]:
data_df=pd.read_csv('Data/Games.csv')

# change size of input from 20 to 100 if using version 2
inputs=np.zeros([300,20]) #[number of inputs, size of each input]
outputs=np.zeros(300) #[number of inputs, result]

In [5]:
inputs,outputs=getData(data_df,inputs,outputs,champ_to_info)

# uncomment to use version 2 inputs and outputs
# inputs,outputs=getData(data_df,inputs,outputs,champ_to_info,version=2)

In [6]:
inputs[0]

array([0.4845, 0.47  , 0.4828, 0.25  , 0.4662, 0.56  , 0.5057, 0.43  ,
       0.4802, 0.4   , 0.4641, 0.53  , 0.5429, 0.53  , 0.5235, 0.77  ,
       0.5148, 0.57  , 0.5208, 0.6   ])

In [7]:
outputs[:10] # 0 represents a win for the first team (loss for the second team), 1 represents a win for the second team (loss for the first team)

array([1., 0., 0., 1., 0., 0., 0., 0., 1., 0.])

# Prepare the data for the network
* Choose which inputs and outputs to use as the training set and testing set (should be 80% training, 20% testing)
* Convert the targets so that they are represented as one-hot vectors when doing categorization

### Training Set

In [8]:
train_x=inputs[:240] # training inputs
train_y=outputs[:240] # training targets

In [9]:
train_x.shape

(240, 20)

In [10]:
train_y.shape

(240,)

### Testing Set

In [11]:
test_x=inputs[240:] # testing inputs
test_y=outputs[240:] #testing targets

In [12]:
test_x.shape

(60, 20)

In [13]:
test_y.shape

(60,)

### Modify Targets

In [14]:
num_categories=2
train_y_category=to_categorical(train_y,num_categories)
test_y_category=to_categorical(test_y,num_categories)

In [15]:
train_y_category[0] # [0. 1.] indicates a win for the second team (loss for the first team)

array([0., 1.], dtype=float32)

# Construct the model

The current network configuration uses 4 hidden layers and a softmax activation function for the output layer to show a degree of “confidence” the network has in the match being a victory or a loss.

In [16]:
ml_nn=Sequential()
ml_nn.add(Dense(75,activation='relu',input_shape=(20,)))
ml_nn.add(Dense(75,activation='relu'))
ml_nn.add(Dense(75,activation='relu'))
ml_nn.add(Dense(75,activation='relu'))
ml_nn.add(Dense(2,activation='softmax'))
ml_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 75)                1575      
                                                                 
 dense_1 (Dense)             (None, 75)                5700      
                                                                 
 dense_2 (Dense)             (None, 75)                5700      
                                                                 
 dense_3 (Dense)             (None, 75)                5700      
                                                                 
 dense_4 (Dense)             (None, 2)                 152       
                                                                 
Total params: 18,827
Trainable params: 18,827
Non-trainable params: 0
_________________________________________________________________


2022-05-08 23:48:06.540705: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 23:48:06.549768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 23:48:06.550051: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 23:48:06.550542: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Compile the model

The current network configuration uses the adam optimizer and the categorical_crossentropy loss function. This loss function interprets the outputs as representing probabilities and forces them to sum to 1.0.  Thus the output from the network will reflect how likely it considers a particular input to be a member of one of the output categories.

In [17]:
# uncomment to update learning rate
# opt=tf.keras.optimizers.Adam(learning_rate=0.0005)

ml_nn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Create an aitk Network
This allows us to do more visualization of what is happening inside the network.

In [23]:
ml_aitk=Network(ml_nn)

In [19]:
from time import sleep
for i in range(10):
    ml_aitk.propagate(train_x[i])
    ml_aitk.display(train_x[i])
    sleep(1.0)

# Train the model

In [ ]:
history=ml_aitk.fit(train_x,train_y_category,verbose=1,validation_data=(test_x,test_y_category),epochs=10)

Epoch 10/10 loss: 0.4031396210193634 - accuracy: 0.8166666626930237 - val_loss: 0.5211445093154907 - val_accuracy: 0.8166666626930237
